### puzzle 1 ###

A "scanner" sweeps back and forth at a constant speed. So, if there are four locations that get checked:

```
  t = 0     t = 1         t = 3      t = 4
 ☒ ☐ ☐ ☐   ☐ ☒ ☐ ☐  ...  ☐ ☐ ☐ ☒    ☐ ☐ ☒ ☐
 0 1 2 3   0 1 2 3       0 1 2 3    0 1 2 3
```
 
So, the scanner position as a function of time (t) and size of sweep (n) can be computed with a little math:

\begin{equation}
t_\text{mod} \equiv t \, \%\, (2n - 2),
\end{equation}

\begin{equation}
x(t) = \begin{cases}
         t_\text{mod}, & t_\text{mod} < n \\
         (2n-2) - t_\text{mod}, & t_\text{mod} \ge n
       \end{cases}
\end{equation}

Note that in the degenerate case of only a single "box", the position is always 0.

In [1]:
def scanner_pos(n, t):
    if n == 0:
        raise ValueError('Size must be > 1')
    elif n == 1:
        return 0
    else:
        tmod = t % (2*n - 2)
        if tmod < n:
            return tmod
        return 2*n - 2 - tmod

For this puzzle, the packet is in layer $t$ at time $t$, always in box 0:

In [2]:
def packet_pos(t, delay=0):
    return t-delay, 0

In [3]:
for t in range(10):
    print(t, scanner_pos(4, t))

0 0
1 1
2 2
3 3
4 2
5 1
6 0
7 1
8 2
9 3


The firewall comprises layers of scanners (each layer has 0 or 1 scanner). The packet starts at layer 0 and traverses the firewall one layer at a time, always in the first "box". The unit of time is how long it takes the packet to traverse a single layer, and also the amount of time that the scanner in a layer moves from one box to the next. If the scanner position for the layer that the packet is in happens to match the box that the packet is in, then there has been a scanner/packet collision. For whatever reason, the severity of the collision is computed as the layer number multiplied by the number of boxes that the scanner is scanning for that layer.

In [4]:
def calc_collision_severity(scanners, delay=0):
    severity_sum = 0
    max_layer = max(scanners.keys())
    # traverse 1 layer in unit time
    for t in range(max_layer + 1 + delay):
        layer, packet_box = packet_pos(t, delay)
        if layer not in scanners:
            continue
        num_boxes = scanners[layer]
        scanner_box = scanner_pos(num_boxes, t)
        if packet_box == scanner_box:
            severity = layer*num_boxes
            severity_sum += severity
    return severity_sum

In [5]:
test_scanners = {0: 3, 1: 2, 4: 4, 6: 4}
assert(24 == calc_collision_severity(test_scanners))

In [6]:
puzzle_lines = open('day13_input').readlines()
puzzle_scanners = {}
for line in puzzle_lines:
    lhs, colon, rhs = line.partition(':')
    puzzle_scanners[int(lhs)] = int(rhs)

In [7]:
calc_collision_severity(puzzle_scanners)

2688

### puzzle 2 ###

Add in a delay before the packet enters the firewall, and look for a delay that leads to no collisions. Unfortunately, the simple method I used in part 1 is way too slow.

Time to encounter layer $i$: $i + d$, where $d$ is the delay.

Iterate over each layer and see if the scanner box at that time is ```packet_box``` (which is always 0).

In [8]:
packet_box = 0
def is_collision(scanners, delay=0):
    for layer, boxes in scanners.items():
        t = layer + delay
        if packet_box == scanner_pos(boxes, t):
            return True
    return False
        

In [9]:
def find_delay_without_collisions(scanners):
    delay = 0
    while True:
        if not is_collision(scanners, delay):
            return delay
        delay += 1

In [10]:
assert(10 == find_delay_without_collisions(test_scanners))

In [11]:
find_delay_without_collisions(puzzle_scanners)

3876272